In [ ]:
%%capture
!pip install --upgrade transformers
!pip install datasets
!pip install rouge_score
!pip install rouge

In [ ]:
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_metric, Dataset
import torch
import numpy as np
import pandas as pd
import io
import math
import time

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/CSC413/amazon_review_dataset_processed.csv"
df = pd.read_csv(path)
amazon = Dataset.from_pandas(df)
amazon.shape

(11848, 3)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      

In [ ]:
task_prefix = "summarize: "
max_source_length = 512
max_target_length = 175

def preprocess_function(reviews):
  input_sequences = reviews['reviewText']
  inputs = [task_prefix + sequence for sequence in input_sequences]
  model_inputs = tokenizer(inputs, max_length=max_source_length, truncation=True, padding=True)

  summaries = reviews['summary']
  labels = tokenizer(summaries, max_length=max_target_length, truncation=True, padding=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_amazon = amazon.map(preprocess_function, batched=True)


NotTest_Test = tokenized_amazon.train_test_split(test_size=0.1, shuffle=True, seed=42)
NotTest = NotTest_Test["train"]
test = NotTest_Test["test"]

Train_Val = NotTest.train_test_split(test_size=0.1, shuffle=True, seed=42)
train = Train_Val["train"]
val = Train_Val["test"]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(torch_device)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_leng

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "./results",
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    fp16=True,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = train,
    eval_dataset = val,
    tokenizer = tokenizer,
    data_collator = data_collator
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend
The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Unnamed: 0, reviewText, summary. If Unnamed: 0, reviewText, summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9596
  Num E

Epoch,Training Loss,Validation Loss
1,1.328800,1.229451
2,1.278900,1.208532
3,1.251800,1.204258


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-2500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-3000] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Unnamed: 0, reviewTex

TrainOutput(global_step=3600, training_loss=1.3595966890123155, metrics={'train_runtime': 2675.0139, 'train_samples_per_second': 10.762, 'train_steps_per_second': 1.346, 'total_flos': 1.753067975344128e+16, 'train_loss': 1.3595966890123155, 'epoch': 3.0})

In [ ]:
trainer.save_model("./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5")

Saving model checkpoint to ./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5
Configuration saved in ./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5/config.json
Model weights saved in ./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5/pytorch_model.bin
tokenizer config file saved in ./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5/tokenizer_config.json
Special tokens file saved in ./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5/special_tokens_map.json


In [ ]:
finetuned = AutoModelForSeq2SeqLM.from_pretrained("./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5")

loading configuration file ./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5/config.json
Model config T5Config {
  "_name_or_path": "./content/drive/MyDrive/Colab Notebooks/CSC413/baseT5",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation

In [ ]:
generated_summaries = []

In [ ]:
x = torch.Tensor(test['input_ids']).long()
output_batch_size = 20
num_of_generate_loops=10

for l in range(num_of_generate_loops):
  print("In loop number: ", l)
  current_batch = x[l*output_batch_size:(l+1)*output_batch_size]
  start = time.time()
  outputs = finetuned.generate(current_batch, max_length=25, min_length=2, num_beams = 2, repetition_penalty = 2.5, early_stopping=True)
  print("Loop : ", l, "took: ", time.time() - start, "to run")
  decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  generated_summaries.append(decoded_outputs)

In loop number:  0
Loop :  0 took:  37.603630781173706 to run
In loop number:  1
Loop :  1 took:  36.88216710090637 to run
In loop number:  2
Loop :  2 took:  37.01695489883423 to run
In loop number:  3
Loop :  3 took:  36.91552209854126 to run
In loop number:  4
Loop :  4 took:  36.81117630004883 to run
In loop number:  5
Loop :  5 took:  36.77254104614258 to run
In loop number:  6
Loop :  6 took:  36.767659187316895 to run
In loop number:  7
Loop :  7 took:  36.907158613204956 to run
In loop number:  8
Loop :  8 took:  36.80144381523132 to run
In loop number:  9
Loop :  9 took:  36.92261624336243 to run


In [ ]:
flattened_outputs = np.array(generated_summaries).flatten()

200

In [ ]:
df_result = pd.DataFrame({'review':test['reviewText'][:200],'generated_summaries':flattened_outputs,'target_summaries':test['summary'][:200]})
df_result.to_csv('./content/drive/MyDrive/Colab Notebooks/CSC413/base_T5_outputs.csv') 

In [ ]:
metric = load_metric('rouge')

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

calc_rouge_scores(flattened_outputs, test['summary'][:200])

{'rouge1': 17.1, 'rouge2': 6.5, 'rougeL': 15.3, 'rougeLsum': 15.3}